In [96]:
import pandas as pd
import csv

In [97]:
tabla_series = pd.read_csv("./components/info_gustos_series.csv", encoding="utf-8")
tabla_movies = pd.read_csv("./components/info_gustos_peliculas.csv", encoding="utf-8")
tabla_musica = pd.read_csv("./components/info_gustos_canciones.csv", encoding="utf-8")
tabla_libros = pd.read_csv("./components/info_gustos_libros.csv", encoding="utf-8")
tabla_juegos = pd.read_csv("./components/info_gustos_videojuegos.csv", encoding="utf-8")
tabla_series.columns = ["Nombre", "Carrera", "Edad", "Series", "Genre"]
tabla_movies.columns = ["Nombre", "Carrera", "Edad", "Pelis", "Genre"]
tabla_musica.columns = ["Nombre", "Carrera", "Edad", "Musica", "Artista", "Genre"]
tabla_libros.columns = ["Nombre", "Carrera", "Edad", "Libro", "Genre"]
tabla_juegos.columns = ["Nombre", "Carrera", "Edad", "Juego"]

peliculas = tabla_movies["Pelis"]
series = tabla_series["Series"]
musica = tabla_musica["Musica"]
libros = tabla_libros["Libro"]
videojuegos = tabla_juegos["Juego"]
videojuegos.pop(0)
print(videojuegos)
# print(libros)
# print(musica)

1         Mario-Kart
2         Mario-Kart
3          Minecraft
4               GTAV
5     Assasins-Creed
           ...      
65         Minecraft
66              Fifa
67         Uncharted
68              Friv
69          Warframe
Name: Juego, Length: 69, dtype: object


In [98]:
# Displaying final output format
from IPython.display import display, Markdown, Latex
# LangChain Dependencies
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
# from langchain_community.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langgraph.graph import START, END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import json
import os

In [99]:
# Defining LLM
local_llm = 'llama3'
llama3 = OllamaLLM(model=local_llm, temperature=0)
llama3_json = OllamaLLM(model=local_llm, format='json', temperature=0)

In [100]:
# Generation Prompt

media_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all tv shows and movies, you will be given an array of may tv shows, and your job is to return
    a your response with the format JSON: {{"NameOfTheShow": "genre"}} with only one genre for each show.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {shows} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["shows"],
)

series_chain = media_analizer | llama3_json | JsonOutputParser()


In [101]:
videogame_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all videogames, you will a videogame, and your job is to return
    a your response with the format JSON: {{"NameOfTheGame": "genre"}} with only one genre for each game.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {videogame} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["videogame"],
)

series_chain = videogame_analizer | llama3_json | JsonOutputParser()

In [102]:
# Generation Prompt

music_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all kind of music, you will be given a song, and your job is to return
    a your response with the format JSON: {{"NameOfTheSong": "genre"}} with only one genre for each song.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {song} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["song"],
)

music_chain = music_analizer | llama3_json | JsonOutputParser()

In [103]:
# Generation Prompt

book_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all kind of books, you will be given a book, and your job is to return
    a your response with the format JSON: {{"NameOfTheBook": "genre"}} with only one genre for each book.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Book: {book} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["book"],
)

book_chain = book_analizer | llama3_json | JsonOutputParser()

In [104]:
# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    print("Step: Generating Final Response")
    shows = state["shows"]
    generation = series_chain.invoke({"shows": shows})

    # Answer Generation

    return {"generation": generation}

In [105]:
# Node - Generate

def generate_games(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Videogames")
    games = state["videogame"]
    generation = series_chain.invoke({"videogame": games})

    # Answer Generation

    return {"generation": generation}

In [106]:
# Node - Generate

def generate_music(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    song = state["song"]
    # print(song)

    # Answer Generation
    generation = music_chain.invoke({"song": song})

    return {"generation": generation}

In [107]:
def generate_books(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    book = state["book"]
    # print(book)

    # Answer Generation
    generation = book_chain.invoke({"book": book})

    return {"generation": generation}

In [108]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    shows: str
    song: str
    book: str
    media: str
    videogame: str
    generation : str

In [109]:
def router(state):
  # print(state["media"])
  return state["media"]

In [110]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_node("generate_shows", generate)
workflow.add_node("generate_music", generate_music)
workflow.add_node("generate_books", generate_books)
workflow.add_node("generate_videogames", generate_games)

workflow.set_conditional_entry_point(
  router,
  {
    "shows": "generate_shows",
    "music": "generate_music",
    "book": "generate_books",
    "videogames": "generate_videogames"
  }
)

workflow.add_edge("generate_shows", END)
workflow.add_edge("generate_music", END)
workflow.add_edge("generate_books", END)
workflow.add_edge("generate_videogames", END)

# Compile the workflow
local_agent = workflow.compile()


In [111]:
def run_agent_series():
    shows = []
    for serie in series:
        output = local_agent.invoke({"media": "shows", "media": "shows", "shows": serie})
        print(output["generation"])
        shows.append(output["generation"])
    return shows

def run_agent_movies():
    movies = []
    for movie in peliculas:
        output = local_agent.invoke({"media": "shows", "shows": movie})
        print(output["generation"])
        movies.append(output["generation"])

    return movies    

def run_agent_music():
    music = []
    for song in musica:
        output = local_agent.invoke({"media": "music", "song": song})
        print(output["generation"])
        music.append(output["generation"])
    
    return music

def run_agent_book():
    books = []
    for book in libros:
        output = local_agent.invoke({"media": "book", "book": book})
        print(output["generation"])
        books.append(output["generation"])
    
    return books

def run_agent_videogames():
    games = []
    for game in videojuegos:
        output = local_agent.invoke({"media": "videogames", "videogame": game})
        print(output["generation"])
        games.append(output["generation"])
    return games

In [112]:
# all_shows = run_agent_series()

In [113]:
# all_movies = run_agent_movies()

In [114]:
# all_music = run_agent_music()

In [115]:
# all_books = run_agent_book()

In [116]:
all_videogames = run_agent_videogames()
# for game in videojuegos:
#   print(game)
#   print(local_agent.invoke({"media": "videogames", "videogame": game})["generation"])

{'Mario Kart': 'Racing'}
{'Mario Kart': 'Racing'}
{'Minecraft': 'Sandbox'}
{'Grand Theft Auto V': 'Action-Adventure'}
{'Assassins Creed': 'Action-Adventure'}
{'Elden Ring': 'Action'}
{'PAYDAY 2': 'Action'}
{'Ark-Survival-Evolved': 'Survival'}
{'The Legend of Zelda': 'Action-Adventure'}
{'Skyrim': 'Action'}
{'Ghost of Tsushima': 'Action-Adventure'}
{'Spider-Man': 'Action'}
{'Minecraft': 'Sandbox'}
{'Subway Surfers': 'Endless Runner'}
{'The Sims': 'Life Simulation'}
{'Roblox': 'Sandbox'}
{'Ark-Survival-Evolved': 'Survival'}
{'Gears of War 3': 'Action'}
{'Marvel-Rivals': 'Fighting'}
{'Halo: Reach': 'First-Person Shooter'}
{'Hell-Let-Loose': 'Action'}
{'Voices-of-the-Void': 'Survival Horror'}
{'Metro 2033': 'First-Person Shooter'}
{'Disco Elysium': 'RPG'}
{"Assassin's Creed Origins": 'Action-Adventure'}
{'God of War': 'Action-Adventure'}
{'Red Dead Redemption 2': 'Western'}
{'The Witcher 3': 'Action'}
{'Life is Strange': 'Adventure'}
{'Halo 3': 'First-Person Shooter'}
{'Batman Arkham City'

In [117]:
df = pd.DataFrame([{"Book": k, "Genre": v} for s in all_videogames for k, v in s.items()])
print(df)
df.to_csv("dict_games.csv", index=False)


                  Book             Genre
0           Mario Kart            Racing
1           Mario Kart            Racing
2            Minecraft           Sandbox
3   Grand Theft Auto V  Action-Adventure
4      Assassins Creed  Action-Adventure
..                 ...               ...
64           Minecraft           Sandbox
65                Fifa            Sports
66           Uncharted  Action-Adventure
67                Friv            Casual
68            Warframe            Action

[69 rows x 2 columns]


In [118]:
# df = pd.DataFrame([{"Show": k, "Genre": v} for s in all_shows for k, v in s.items()])
# print(df)
# df.to_csv("dict_series.csv", index=False)
# df = pd.DataFrame([{"Movie": k, "Genre": v} for s in all_movies for k, v in s.items()])
# print(df)
# df.to_csv("dict_movies.csv", index=False)

In [119]:
#References:
# Pandas: https://pandas.pydata.org/docs/getting_started/index.html
# Langchain: https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining 